In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization
from isic_helper import compute_pauc, compute_auc

In [2]:
def blend_optimizer(oof_preds_df, oof_columns, folds, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (1e-5, 1.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights

In [3]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_ensemble"
fold_column = "gkf_fold"

boosting_model_names = ["xgb", "xgb", "lgb", "cb"]
boosting_versions = ["v4", "v5", "v6", "v1"]
boosting_modes = ["train", "train", "train", "train"]

cnn_model_names = ["efficientnet_b1", "efficientnet_b2"]
cnn_versions = ["v1", "v1"]
cnn_modes = ["trainbinary", "trainbinary"]

model_names = boosting_model_names + cnn_model_names
versions = boosting_versions + cnn_versions
modes = boosting_modes + cnn_modes

paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

val_auc_scores = {}
val_pauc_scores = {}
oof_columns = []
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    mode = modes[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_name}_{version}.csv")
    if mode in ["trainbinary", "trainmulti"]:
        oof_column = f"oof_{model_name}_{version}_{mode}"
        oof_preds_model_df = oof_preds_model_df.rename(columns={f"oof_{model_name}_{version}": oof_column})
    else:
        oof_column = f"oof_{model_name}_{version}"
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, oof_column]], on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    
    val_auc_scores[f"{model_name}_{version}_{mode}"] = {}
    val_pauc_scores[f"{model_name}_{version}_{mode}"] = {}
    oof_columns.append(oof_column)

all_folds = np.unique(oof_preds_df["fold"])
weights = blend_optimizer(
    oof_preds_df, oof_columns, all_folds,
    init_points=50, n_iter=100
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |    w4     |    w5     |
-------------------------------------------------------------------------------------------------
| 1         | 0.1814    | 0.009369  | 0.4991    | 0.1134    | 0.04998   | 0.6854    | 0.487     |
| 2         | 0.1839    | 0.8977    | 0.6475    | 0.897     | 0.7211    | 0.8314    | 0.8276    |
| 3         | 0.1831    | 0.8336    | 0.957     | 0.3681    | 0.4948    | 0.3395    | 0.6194    |
| 4         | 0.1834    | 0.9775    | 0.09644   | 0.7442    | 0.2925    | 0.2987    | 0.7525    |
| 5         | 0.184     | 0.01867   | 0.5237    | 0.8644    | 0.3888    | 0.2122    | 0.4752    |
| 6         | 0.1838    | 0.5647    | 0.3494    | 0.9759    | 0.03783   | 0.7943    | 0.3579    |
| 7         | 0.1833    | 0.748     | 0.9145    | 0.3727    | 0.9649    | 0.08139   | 0.04246   |
| 8         | 0.1838    | 0.2968    | 0.3637    | 0.4903    | 0.6685    | 0.6734    | 0.5721    |
| 9         | 0.1831

In [4]:
weights

[0.34918300064067676,
 0.40612525924227244,
 0.563925272769909,
 1.0,
 0.4360364110294176,
 0.11920415742858939]

In [5]:
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, mode, oof_column, weight in zip(model_names, versions, modes, oof_columns, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, oof_column]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}_{mode}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}_{mode}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, mode, oof_column, weight in zip(model_names, versions, modes, oof_columns, weights):
    print(f"Model: {model_name}_{version}_{mode} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}_{mode}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}_{mode}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[oof_column])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[oof_column], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}_{mode}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}_{mode}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}_{mode}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}_{mode}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")
print(f"CV PAUC RANGE: ({cv_ensemble_pauc_avg - cv_ensemble_pauc_std}, {cv_ensemble_pauc_avg + cv_ensemble_pauc_std})")

Model: xgb_v4_train | Weightage: 0.34918300064067676
Val AUC scores:
{'fold_1': 0.9774012004442393,
 'fold_2': 0.9585113204613043,
 'fold_3': 0.9635616209706606,
 'fold_4': 0.9713384957132696,
 'fold_5': 0.9692148821814502}
Val PAUC scores:
{'fold_1': 0.18439364147339235,
 'fold_2': 0.16635088150424956,
 'fold_3': 0.17198002333678597,
 'fold_4': 0.174545119866219,
 'fold_5': 0.17352956799065142}
CV AUC OOF: 0.947856723136131
CV PAUC OOF: 0.1540868271122474
CV AUC AVG: 0.9680055039541848
CV PAUC AVG: 0.17415984683425967
CV AUC STD: 0.006492120739229583
CV PAUC STD: 0.005848210144159227


Model: xgb_v5_train | Weightage: 0.40612525924227244
Val AUC scores:
{'fold_1': 0.9800866989306786,
 'fold_2': 0.9563666755620234,
 'fold_3': 0.9702503556675942,
 'fold_4': 0.9640259387752556,
 'fold_5': 0.9668575032470057}
Val PAUC scores:
{'fold_1': 0.18589260048011932,
 'fold_2': 0.1629143620333377,
 'fold_3': 0.1782176997666321,
 'fold_4': 0.17031127154284853,
 'fold_5': 0.17059913534774074}
CV AUC 

In [6]:
oof_preds_df.pivot_table(index="target", values=oof_columns + [ensemble_column], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b1_v1_trainbinary,oof_efficientnet_b2_v1_trainbinary,oof_ensemble,oof_lgb_v6,oof_xgb_v4,oof_xgb_v5
target,,,,,,,
0,0.009339,0.000992,0.000783,1.435923,0.010528,0.014427,0.022984
1,0.423538,0.230465,0.211758,2.795299,0.455325,0.329018,0.345672


In [7]:
oof_preds_df[oof_columns + [ensemble_column]].corr()

,oof_xgb_v4,oof_xgb_v5,oof_lgb_v6,oof_cb_v1,oof_efficientnet_b1_v1_trainbinary,oof_efficientnet_b2_v1_trainbinary,oof_ensemble
oof_xgb_v4,1.000000,0.778952,0.711025,0.674799,0.403045,0.387978,0.324067
oof_xgb_v5,0.778952,1.000000,0.655779,0.632031,0.357309,0.347570,0.333691
oof_lgb_v6,0.711025,0.655779,1.000000,0.927002,0.458106,0.422038,0.353773
oof_cb_v1,0.674799,0.632031,0.927002,1.000000,0.495410,0.454357,0.343308
oof_efficientnet_b1_v1_trainbinary,0.403045,0.357309,0.458106,0.495410,1.000000,0.890838,0.089547
oof_efficientnet_b2_v1_trainbinary,0.387978,0.347570,0.422038,0.454357,0.890838,1.000000,0.081071
oof_ensemble,0.324067,0.333691,0.353773,0.343308,0.089547,0.081071,1.000000
